# Baseline Forecast

In [1]:
from darts.models.forecasting.baselines import LocalForecastingModel

/Users/nayeongkim/miniconda3/envs/HKweather3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/nayeongkim/miniconda3/envs/HKweather3.9/lib/python3.9/site-packages/statsforecast/utils.py:231: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),
The StatsForecast module could not be imported. To enable support for the StatsForecastAutoARIMA, StatsForecastAutoETS and Croston models, please consider installing it.


In [2]:
import sys
sys.path.append('/Users/nayeongkim/Desktop/HKweather/')
print(sys.path)
import pandas as pd
from functions.eval import evaluate_performance
from functions import ts_utils
from functions.plot_functions import plot_forecast
from functions.plot_functions import format_plot
from functions.forecast_bs import NaiveMovingAverage

['/Users/nayeongkim/miniconda3/envs/HKweather3.9/lib/python39.zip', '/Users/nayeongkim/miniconda3/envs/HKweather3.9/lib/python3.9', '/Users/nayeongkim/miniconda3/envs/HKweather3.9/lib/python3.9/lib-dynload', '', '/Users/nayeongkim/miniconda3/envs/HKweather3.9/lib/python3.9/site-packages', '/Users/nayeongkim/miniconda3/envs/HKweather3.9/lib/python3.9/site-packages/setuptools/_vendor', '/Users/nayeongkim/Desktop/HKweather/']


In [3]:
daily_temp_train = pd.read_parquet("../0_Data/wrangled/daily_temp_train.parquet")
daily_temp_val = pd.read_parquet("../0_Data/wrangled/daily_temp_val.parquet")
daily_temp_test = pd.read_parquet("../0_Data/wrangled/daily_temp_test.parquet")

In [4]:
# Data Frequency setting
freq = "D" # daily frequency (not even the business day frequency)

In [5]:
daily_temp_train.reset_index(inplace = True)
daily_temp_test.reset_index(inplace = True)
daily_temp_val.reset_index(inplace = True)

In [6]:
daily_temp_train.rename(columns = {'Date':'ds'}, inplace = True)
daily_temp_test.rename(columns = {'Date':'ds'}, inplace = True)
daily_temp_val.rename(columns = {'Date':'ds'}, inplace = True)

In [7]:
pred_df = pd.concat([daily_temp_train, daily_temp_val])

In [8]:
from statsforecast import StatsForecast
from utilsforecast.evaluation import evaluate
from statsforecast.models import (
    Naive,
    SeasonalNaive,
    HistoricAverage,
    WindowAverage,
    SeasonalWindowAverage,
    RandomWalkWithDrift,
    HoltWinters,
    ETS,
    AutoETS,
    AutoARIMA,
    #ARIMA,
    AutoTheta,
    DynamicTheta,
    DynamicOptimizedTheta,
    Theta,
    OptimizedTheta,
    #TBATS,
    #AutoTBATS,
    MSTL
)

- **Naïve forecast**: Proposing the last/most recent observation in a time series as a forecast
- **Moving average forecast**: A moving average forecast is another simple method that tries to overcome the pure memorization of the naïve method. Instead of taking the latest observation, it takes the mean of the latest n steps as the forecast.

In [9]:
daily_temp_train.tail(1)

,ds,y,unique_id
3652,2024-12-31,23.454054,1.0


In [10]:
daily_temp_train

,ds,y,unique_id
0,2015-01-01,20.161111,1.0
1,2015-01-02,17.941667,1.0
2,2015-01-03,19.447222,1.0
3,2015-01-04,21.361111,1.0
4,2015-01-05,21.580556,1.0
...,...,...,...
3648,2024-12-27,21.005405,1.0
3649,2024-12-28,19.089189,1.0
3650,2024-12-29,18.137838,1.0
3651,2024-12-30,22.337838,1.0


In [30]:
from functions.ts_utils import mase, mae, mse, forecast_bias

metrics = pd.DataFrame()

results, metrics = evaluate_performance(
    ts_train=daily_temp_train, 
    ts_test=daily_temp_val, 
    models=[Naive()], 
    metrics=[mase, mae, mse, forecast_bias], 
    freq=freq,
    level=[],  # Ensure this is correct or adjust as necessary
    id_col='unique_id',
    time_col='ds',
    target_col='y',
    h=len(daily_temp_val),
    metric_df=metrics  # Pass None or an existing DataFrame if you want to append results
)

             ds          y  unique_id
0    2015-01-01  20.161111        1.0
1    2015-01-02  17.941667        1.0
2    2015-01-03  19.447222        1.0
3    2015-01-04  21.361111        1.0
4    2015-01-05  21.580556        1.0
...         ...        ...        ...
3648 2024-12-27  21.005405        1.0
3649 2024-12-28  19.089189        1.0
3650 2024-12-29  18.137838        1.0
3651 2024-12-30  22.337838        1.0
3652 2024-12-31  23.454054        1.0

[3653 rows x 3 columns]
           ds          y  unique_id      Naive
0  2025-01-01  18.405405        1.0  23.454054
1  2025-01-02  22.029730        1.0  23.454054
2  2025-01-03  21.448649        1.0  23.454054
3  2025-01-04  20.354054        1.0  23.454054
4  2025-01-05  22.477778        1.0  23.454054
5  2025-01-06  22.300000        1.0  23.454054
6  2025-01-07  19.862162        1.0  23.454054
7  2025-01-08  21.464865        1.0  23.454054
8  2025-01-09  21.116216        1.0  23.454054
9  2025-01-10  17.316216        1.0  23.454054
10

/Users/nayeongkim/miniconda3/envs/HKweather3.9/lib/python3.9/site-packages/numpy/core/numeric.py:330: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')


In [31]:
results.head()

,ds,y,unique_id,Naive
0,2025-01-01,18.405405,1.0,23.454054
1,2025-01-02,22.029730,1.0,23.454054
2,2025-01-03,21.448649,1.0,23.454054
3,2025-01-04,20.354054,1.0,23.454054
4,2025-01-05,22.477778,1.0,23.454054


In [13]:
metrics

,mase,mae,mse,forecast_bias,unique_id,Time Elapsed,Model
0,2.084903,3.269069,13.360225,-16.19553,1.0,0 days 00:00:01.425804,Naive


In [ ]:
model_name = ['Naive']
model_display_name = ['Naive']

fig = plot_forecast(results, forecast_columns=model_name, forecast_display_names=model_display_name, timestamp_col ='ds')
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.show()

In [34]:
from darts import TimeSeries
train_TS = TimeSeries.from_dataframe(daily_temp_train, time_col = "ds", value_cols="y")

In [35]:
naive_model = NaiveMovingAverage(window=48)
naive_model.fit(train_TS)
pred_MA = naive_model.predict(31)
results['bs_MA'] = pred_MA.univariate_values()

model_name = ['bs_MA']
model_display_name = ['bs_MA']

In [36]:
id_col = 'unique_id'
target_col = 'y'
model_name = 'bs_MA'
ids = daily_temp_val[id_col].unique()
metrics = [mase, mae, mse, forecast_bias]
evaluation = {} 
metric_df = pd.DataFrame()

# Calculate metrics

for id in ids:
    
    temp_results = results[results[id_col] == id]
    temp_train = daily_temp_train[daily_temp_train[id_col] == id]

    for metric in metrics:
        metric_name = metric.__name__
        if metric_name == 'mase':
            evaluation[metric_name] = metric(temp_results[target_col].values,
                                             temp_results[model_name].values,
                                             temp_train[target_col].values)
        else:
            evaluation[metric_name] = metric(temp_results[target_col].values, temp_results[model_name].values)
    evaluation[id_col] = id

    # Prepare and append this model's results to metric_df
    temp_df = pd.DataFrame(evaluation, index=[0])
    temp_df['Model'] = model_name
    metric_df = pd.concat([metric_df, temp_df], ignore_index=True)

In [37]:
metrics = metric_df

In [39]:
model_name = ['bs_MA']
model_display_name = ['bs_MA']

fig = plot_forecast(results, 
                    forecast_columns=model_name, 
                    forecast_display_names=model_display_name,
                    timestamp_col ='ds')

In [ ]:
fig = format_plot(fig, title=f"{model_name[0]}: "\
                  f"MAE: {metrics.loc[metrics.Model==model_name[0]][['mae']].iloc[0].item():.4f} | "\
                  f"MASE: {metrics.loc[metrics.Model==model_name[0]][['mase']].iloc[0].item():.4f} | "\
                  f"BIAS: {metrics.loc[metrics.Model==model_name[0]][['forecast_bias']].iloc[0].item():.4f}")
fig.show()